In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv("smsspamcollection.tsv", sep='\t')

In [3]:
df.head()

,label,message,length,punct
0,ham,"Go until jurong point, crazy.. Available only ...",111,9
1,ham,Ok lar... Joking wif u oni...,29,6
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,155,6
3,ham,U dun say so early hor... U c already then say...,49,6
4,ham,"Nah I don't think he goes to usf, he lives aro...",61,2


In [4]:
df.isnull().sum() # check for null values

label      0
message    0
length     0
punct      0
dtype: int64

In [5]:
df["label"].value_counts()

ham     4825
spam     747
Name: label, dtype: int64

In [6]:
from sklearn.model_selection import train_test_split

In [7]:
X = df["message"]

In [8]:
y = df["label"]

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.33, random_state=42)

## Count vectorization

In [10]:
from sklearn.feature_extraction.text import CountVectorizer

In [11]:
count_vec = CountVectorizer()

In [12]:
X_train_counts = count_vec.fit_transform(X_train)

In [13]:
X_train_counts

<3733x7082 sparse matrix of type '<class 'numpy.int64'>'
	with 49992 stored elements in Compressed Sparse Row format>

In [14]:
X_train.shape

(3733,)

In [15]:
X_train_counts.shape

(3733, 7082)

## TF-IDF : term frequency - inverse document frequency

In [16]:
from sklearn.feature_extraction.text import TfidfTransformer

In [17]:
tfidfTransformer = TfidfTransformer()

In [18]:
X_train_tfidf = tfidfTransformer.fit_transform(X_train_counts)

In [19]:
X_train_tfidf.shape

(3733, 7082)

### Count vectorization and TFIDF both can perform by,

In [20]:
'''
from sklearn.feature_extraction.text import TfidfVectorizer
tfidfVectorizer = TfidfTransformer()
X_train_tfidf = tfidfVectorizer.fit_transform(X_train)
'''

'\nfrom sklearn.feature_extraction.text import TfidfVectorizer\ntfidfVectorizer = TfidfTransformer()\nX_train_tfidf = tfidfVectorizer.fit_transform(X_train)\n'

In [21]:
from sklearn.svm import LinearSVC

In [22]:
clf = LinearSVC()

In [23]:
clf.fit(X_train_tfidf, y_train)

LinearSVC()

## Use pipeline for simplify steps

In [26]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [27]:
from sklearn.pipeline import Pipeline
text_clf = Pipeline([('tfidf', TfidfVectorizer()), ('clf', LinearSVC())])
text_clf.fit(X_train, y_train)  

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', LinearSVC())])

In [28]:
y_pred = text_clf.predict(X_test)

In [34]:
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

In [31]:
print(confusion_matrix(y_pred, y_test))

[[1586   12]
 [   7  234]]


In [32]:
print(classification_report(y_pred, y_test))

              precision    recall  f1-score   support

         ham       1.00      0.99      0.99      1598
        spam       0.95      0.97      0.96       241

    accuracy                           0.99      1839
   macro avg       0.97      0.98      0.98      1839
weighted avg       0.99      0.99      0.99      1839



In [35]:
print(accuracy_score(y_pred, y_test))

0.989668297988037


In [37]:
pred = text_clf.predict(["Hi how are you doing today?"])

In [38]:
pred

array(['ham'], dtype=object)

In [41]:
pred = text_clf.predict(["Congratulations! you are selected winner. TEXT WIN to 221 for get your free PASS"])

In [42]:
pred

array(['spam'], dtype=object)